In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib inline
import matplotlib.pylab as plt

In [18]:
from matplotlib import pylab
pylab.rcParams['figure.figsize'] = (5, 2.5)

In [3]:
import ld_gan
import numpy as np

# load data

In [4]:
X, Y = ld_gan.data_proc.data_loader.load_data(1, verbose=1, resize = 64)
n_classes = Y.shape[1]
Y = np.argmax(Y, axis = 1)

100%|██████████| 102/102 [00:40<00:00,  2.63it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_102/jpg_128'


# load models

In [5]:
project = "xf_11181110.py"
epoch = 100

In [6]:
enc = ld_gan.utils.model_handler.load_parallel_model(project, epoch, "enc")
gen = ld_gan.utils.model_handler.load_parallel_model(project, epoch, "gen")
dis = ld_gan.utils.model_handler.load_parallel_model(project, epoch, "dis")

loaded model 'projects/xf_11181110.py/model/e_0100.pth'
loaded model 'projects/xf_11181110.py/model/g_0100.pth'
loaded model 'projects/xf_11181110.py/model/d_0100.pth'


# best_in_pos

In [13]:
# choose 10 ancs
x_ancs = X[np.random.randint(0, len(X), 2)]

In [14]:
z_anc = ld_gan.utils.model_handler.apply_model(enc, x_ancs)
z_all = ld_gan.utils.model_handler.apply_model(enc, X, 100)

In [26]:
z_anc.reshape()

In [51]:
out = ld_gan.train_ops.triplet_enc.generate_triplets(enc, gen, dis, x_ancs, X, 7, 
                                               n_neg=5, n_pos=5)

In [ ]:
(7, 2, 10, 256)

In [9]:
idxs_pos, idxs_neg = ld_gan.train_ops.triplet_enc.get_enc_space_suggestion(
                                                            z_anc, 
                                                            z_all, 
                                                            n_pos=10, 
                                                            n_neg=150, 
                                                            quantile_pos=0.01, 
                                                            quantile_neg=0.01)

In [33]:
can_pos = z_all[idxs_pos]
can_neg = z_all[idxs_neg]

In [36]:
can_pos.shape, can_neg.shape

((10, 10, 256), (10, 150, 256))

In [30]:
ds_pos = ld_gan.train_ops.triplet_enc.get_dis_scores(gen, dis, z_anc, can_pos)

In [31]:
ds[0]

array([  4.25695005e-04,   1.55771419e-03,   3.67129128e-03,
         3.27814952e-03,   1.55772595e-03,   5.80587075e-05,
         1.86518778e-03,   1.37790910e-03,   5.23491204e-03,
         3.04996874e-03], dtype=float32)

In [32]:
ds[0].max()

0.005234912

In [42]:
ds[0].mean()

0.002207661

In [39]:
ds_neg = ld_gan.train_ops.triplet_enc.get_dis_scores(gen, dis, z_anc, can_neg)

In [41]:
ds_neg[0].max()

0.017589888

In [43]:
ds_neg[0].mean()

0.0033766404

In [ ]:
    zs_new = []
    for i0 in range(zs.shape[0]):
        for i1 in range(zs.shape[1]):
            for i2 in range(zs.shape[2]):
                zs_new.append(zs[i0, i1, i2])
    zs_new = np.array(zs_new)

In [ ]:
    ds_new = np.zeros(ds_shape)
    idx_jj = 0
    for i0 in range(ds_shape[0]):
        for i1 in range(ds_shape[1]):
            for i2 in range(ds_shape[2]):
                ds_new[i0, i1, i2] = ds[idx_jj]
                idx_jj += 1